# Python 上下文管理器 (`with` 语句) 与 `contextlib` 模块教程

欢迎来到 Python 上下文管理器和 `contextlib` 模块的教程！`with` 语句是 Python 中一种优雅且强大的资源管理方式，它可以确保某些操作（如文件关闭、锁的释放）即使在发生异常时也能正确执行。本教程将深入探讨上下文管理器的工作原理、如何自定义上下文管理器，以及如何使用 `contextlib` 模块中的工具来更轻松地创建它们。

**为什么使用 `with` 语句和上下文管理器？**

1.  **资源管理**：自动获取和释放资源（如文件、网络连接、数据库会话、锁等），确保即使发生错误，清理代码（通常在 `finally` 块中）也能被执行。
2.  **代码简洁性**：避免了显式的 `try...finally` 结构，使代码更易读。
3.  **封装设置和拆卸逻辑**：将资源的准备和清理逻辑封装在上下文管理器内部。

**本教程将涵盖：**

1.  **回顾 `with` 语句的基本用法**
2.  **上下文管理协议 (Context Management Protocol)**：`__enter__` 和 `__exit__`
3.  **创建自定义上下文管理器 (基于类)**
4.  **`contextlib` 模块简介**
    *   `@contextlib.contextmanager` 装饰器：使用生成器创建上下文管理器
    *   `contextlib.closing`：确保对象的 `close()` 方法被调用
    *   `contextlib.suppress`：临时抑制指定的异常
    *   `contextlib.ExitStack` 和 `AsyncExitStack`：动态管理多个上下文管理器
5.  **异步上下文管理器 (Async Context Managers)**：`__aenter__` 和 `__aexit__` (简要回顾)
6.  **实际应用场景**

## 1. 回顾 `with` 语句的基本用法

你最常看到的 `with` 语句可能是用于文件操作：

In [ ]:
file_path = "example_with_statement.txt"

# 使用 with 语句打开文件
try:
    with open(file_path, "w") as f:
        print(f"File object type inside with: {type(f)}")
        f.write("Hello from the with statement!\n")
        print("Wrote to file.")
        # 文件 f 会在退出 with 块时自动关闭，即使发生异常
        # raise ValueError("Simulating an error inside with") # 取消注释测试异常处理
    print(f"File '{file_path}' closed automatically: {f.closed}")
except ValueError as e:
    print(f"Caught error: {e}")
    # 即使发生错误，文件仍然会被关闭
    # 注意：此时 f 可能未定义，取决于异常发生的位置
    # print(f"File closed after error: {f.closed}") # 这行可能会报错
    # 更好的做法是在 finally 中检查，或者信任 with 语句
    pass 

# 清理文件
import os
if os.path.exists(file_path):
    os.remove(file_path)

这里的 `open()` 函数返回的文件对象就是一个**上下文管理器 (context manager)**。

## 2. 上下文管理协议 (Context Management Protocol)

任何实现了以下两个特殊方法的对象都可以作为上下文管理器：

*   **`__enter__(self)`**：
    *   在进入 `with` 语句块之前被调用。
    *   它的返回值（如果有的话）会赋给 `with ... as variable:` 中的 `variable`。
    *   如果不需要在 `as` 子句中接收值，`__enter__` 可以不返回任何东西（即返回 `None`）。

*   **`__exit__(self, exc_type, exc_val, exc_tb)`**：
    *   在退出 `with` 语句块时被调用，无论块内代码是正常完成还是发生异常。
    *   参数：
        *   `exc_type`: 异常的类型（如果块内没有异常，则为 `None`）。
        *   `exc_val`: 异常的实例（如果块内没有异常，则为 `None`）。
        *   `exc_tb`: 回溯 (traceback) 对象（如果块内没有异常，则为 `None`）。
    *   **返回值**：
        *   如果 `__exit__` 方法返回 `True`，则表示它已经处理了异常，异常会被“抑制”，不会向外传播。
        *   如果 `__exit__` 方法返回 `False` (或 `None`，这是默认行为)，则发生的任何异常都会在 `__exit__` 方法执行完毕后重新引发，由外部代码处理。

## 3. 创建自定义上下文管理器 (基于类)

我们可以通过定义一个包含 `__enter__` 和 `__exit__` 方法的类来创建自己的上下文管理器。

In [ ]:
import time

class Timer:
    """一个简单的计时器上下文管理器"""
    def __init__(self, name="Default Timer"):
        self.name = name
        self.start_time = None
        print(f"Timer '{self.name}': Initialized.")

    def __enter__(self):
        print(f"Timer '{self.name}': __enter__ - Starting timer.")
        self.start_time = time.perf_counter()
        return self # 返回自身，以便在 with 块内可以访问 Timer 实例的属性 (可选)

    def __exit__(self, exc_type, exc_val, exc_tb):
        end_time = time.perf_counter()
        elapsed_time = end_time - self.start_time
        print(f"Timer '{self.name}': __exit__ - Elapsed time: {elapsed_time:.4f} seconds.")
        if exc_type:
            print(f"  Timer '{self.name}': Exception occurred: {exc_type.__name__} - {exc_val}")
        # return False # 默认行为，不抑制异常

print("--- Testing Timer context manager (normal execution) ---")
with Timer("MyOperation") as t:
    print(f"  Inside with block for timer '{t.name}'. Accessing start_time (approx): {t.start_time}")
    time.sleep(0.5)
    print("  Work inside with block done.")

print("\n--- Testing Timer context manager (with exception) ---")
try:
    with Timer("RiskyOperation") as risky_t:
        print(f"  Inside with block for timer '{risky_t.name}'.")
        time.sleep(0.2)
        raise KeyError("Simulated key error!")
except KeyError as e:
    print(f"Caught expected KeyError outside 'with': {e}")

# 示例：抑制异常
class SuppressErrorExample:
    def __enter__(self):
        print("SuppressErrorExample: Entering")
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("SuppressErrorExample: Exiting")
        if isinstance(exc_val, TypeError):
            print(f"  Suppressing TypeError: {exc_val}")
            return True # 抑制 TypeError
        return False # 其他异常不抑制

print("\n--- Testing SuppressErrorExample (suppressing TypeError) ---")
with SuppressErrorExample():
    print("  Trying to cause a TypeError...")
    _ = "text" + 5 # This will raise TypeError
print("After with block (TypeError was suppressed)")

print("\n--- Testing SuppressErrorExample (not suppressing ValueError) ---")
try:
    with SuppressErrorExample():
        print("  Trying to cause a ValueError...")
        raise ValueError("This error won't be suppressed")
except ValueError as e:
    print(f"Caught ValueError outside 'with' as expected: {e}")

## 4. `contextlib` 模块简介

`contextlib` 模块提供了一些实用工具，用于处理上下文管理器和 `with` 语句。

### 4.1 `@contextlib.contextmanager` 装饰器

这个装饰器允许你使用一个简单的**生成器函数**来创建上下文管理器，而无需编写一个完整的类。

**要求：**
*   被装饰的生成器函数必须**只 `yield` 一次**。
*   `yield` 之前的部分代码相当于 `__enter__` 方法的逻辑。
*   `yield` 语句产生的值会赋给 `with ... as variable:` 中的 `variable`。
*   `yield` 之后的部分代码（通常在 `try...finally` 块中）相当于 `__exit__` 方法的逻辑，确保即使发生异常也能执行清理。
*   生成器内部发生的异常会正常传播，除非在 `finally` 块中被捕获或处理。

In [ ]:
import contextlib

@contextlib.contextmanager
def managed_file(filename, mode):
    print(f"@contextmanager: Acquiring file '{filename}' in mode '{mode}'")
    f = None
    try:
        f = open(filename, mode)
        yield f # 这是 __enter__ 返回的值，也是生成器暂停的地方
        print(f"@contextmanager: Inside try block after yield (normal exit of 'with' body)")
    except Exception as e:
        print(f"@contextmanager: Exception caught inside generator: {e}")
        raise # 重新引发异常，除非你想抑制它
    finally:
        if f:
            print(f"@contextmanager: Releasing file '{filename}' (in finally)")
            f.close()

file_path_cm = "example_cm.txt"

print("--- Testing @contextmanager (normal execution) ---")
with managed_file(file_path_cm, "w") as mf:
    print(f"  Inside with: File object is {mf}")
    mf.write("Hello from @contextmanager!")
    print(f"  Inside with: File closed status: {mf.closed}") # 应该是 False
print(f"After with: Is file closed? {mf.closed}") # 应该是 True

print("\n--- Testing @contextmanager (with exception) ---")
try:
    with managed_file(file_path_cm, "r") as mf_read: # 应该能读到刚才写的内容
        print(f"  Inside with: Reading content: {mf_read.read().strip()}")
        raise ZeroDivisionError("Simulated error for @contextmanager")
except ZeroDivisionError as e:
    print(f"Caught expected ZeroDivisionError outside 'with': {e}")

# 清理
if os.path.exists(file_path_cm):
    os.remove(file_path_cm)

### 4.2 `contextlib.closing(thing)`

如果一个对象 `thing` 提供了 `close()` 方法但没有实现上下文管理协议（即没有 `__enter__` 和 `__exit__`），`closing(thing)` 会返回一个上下文管理器，它在退出时调用 `thing.close()`。

这对于处理一些旧式的、只提供 `close()` 方法的资源很有用。

In [ ]:
class OldResource:
    def __init__(self, name):
        self.name = name
        self.is_closed = False
        print(f"OldResource '{self.name}': Created.")
    
    def use(self):
        if self.is_closed:
            raise ValueError("Resource is closed")
        print(f"  OldResource '{self.name}': Being used.")
        
    def close(self):
        print(f"OldResource '{self.name}': close() called.")
        self.is_closed = True

print("--- Testing contextlib.closing --- ")
resource = OldResource("LegacyDB")

with contextlib.closing(resource) as r_closed:
    # r_closed 就是 resource 本身
    print(f"  Inside with: resource is r_closed: {resource is r_closed}")
    r_closed.use()
print(f"After with: resource.is_closed = {resource.is_closed}") # 应该是 True

print("\n--- Testing contextlib.closing with an error --- ")
resource2 = OldResource("LegacySocket")
try:
    with contextlib.closing(resource2) as r2_closed:
        r2_closed.use()
        raise RuntimeError("Error while using legacy resource")
except RuntimeError as e:
    print(f"Caught expected RuntimeError: {e}")
print(f"After with (error): resource2.is_closed = {resource2.is_closed}") # 仍然应该是 True

### 4.3 `contextlib.suppress(*exceptions)`

返回一个上下文管理器，用于临时抑制指定的异常类型。在 `with` 块中，如果发生了列出的异常类型，它们会被捕获并静默处理，程序会正常继续执行 `with` 块之后的代码。

**注意**：应谨慎使用，确保你确实想要忽略这些异常，而不是隐藏了重要的问题。

In [ ]:
print("--- Testing contextlib.suppress --- ")

print("Attempting to delete a non-existent file (will suppress FileNotFoundError):")
non_existent_file = "no_such_file.txt"
with contextlib.suppress(FileNotFoundError, PermissionError):
    os.remove(non_existent_file)
    print(f"  Code inside 'with' after os.remove attempt.") # 这行会执行
print("After 'with' block, FileNotFoundError was suppressed.")

print("\nAttempting an operation that causes TypeError (will not be suppressed):")
try:
    with contextlib.suppress(FileNotFoundError):
        result = "text" + 10 # Raises TypeError
        print("  This line (inside with) will not be reached if TypeError occurs.")
except TypeError as e:
    print(f"Caught TypeError outside 'with' as expected: {e}")
print("After 'with' block for TypeError.")

### 4.4 `contextlib.ExitStack` 和 `AsyncExitStack`

`ExitStack` 是一个上下文管理器，它允许你以编程方式注册多个上下文管理器或清理函数，并在 `ExitStack` 本身退出时，以注册的相反顺序调用它们的 `__exit__` 方法或清理函数。
这对于动态管理一组不确定数量的资源非常有用。

`AsyncExitStack` 是其异步版本，用于 `async with`。

In [ ]:
class ResourceForStack:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print(f"ResourceForStack '{self.name}': Entering")
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"ResourceForStack '{self.name}': Exiting")
    def use(self):
        print(f"  Using '{self.name}'")

def cleanup_function(resource_name):
    print(f"Cleanup function called for '{resource_name}'")

print("--- Testing contextlib.ExitStack --- ")
resources_to_manage = [
    ResourceForStack("ResA"),
    ResourceForStack("ResB")
]

with contextlib.ExitStack() as stack:
    print("Inside ExitStack 'with' block.")
    
    # 动态进入上下文管理器
    for res_obj in resources_to_manage:
        r = stack.enter_context(res_obj)
        r.use()
    
    # 注册一个简单的清理回调函数
    # stack.callback(cleanup_function, "DynamicResource1")
    # stack.callback(cleanup_function, "DynamicResource2")
    # 或者使用 push (可以 unregister)
    stack.push(lambda: cleanup_function("PushedCleanup1"))
    
    print("Simulating work within ExitStack...")
    # 如果这里发生异常，所有已注册的 __exit__ 和回调仍会以相反顺序调用
    # raise ValueError("Error inside ExitStack") 

print("After ExitStack 'with' block. Resources should be released in reverse order of entry/push.")

## 5. 异步上下文管理器 (Async Context Managers) - 简要回顾

对于异步编程 (`async/await`)，上下文管理器需要使用异步版本的方法：

*   **`__aenter__(self)`**: 必须是一个 `async def` 方法，并且应该 `await` 异步操作。它返回的值（通常是 `awaitable` 解析后的值或 `self`）赋给 `async with ... as var` 中的 `var`。
*   **`__aexit__(self, exc_type, exc_val, exc_tb)`**: 也必须是一个 `async def` 方法。

`contextlib` 也提供了异步版本：
*   `@contextlib.asynccontextmanager`
*   `contextlib.AsyncExitStack`

In [ ]:
import asyncio

@contextlib.asynccontextmanager
async def async_managed_resource(name):
    print(f"AsyncCM '{name}': Acquiring resource (async)...", flush=True)
    await asyncio.sleep(0.1) # 模拟异步获取
    try:
        yield name # 值赋给 as 后面的变量
        print(f"AsyncCM '{name}': Inside try after yield (normal async with body exit).", flush=True)
    finally:
        print(f"AsyncCM '{name}': Releasing resource (async, in finally)...", flush=True)
        await asyncio.sleep(0.1) # 模拟异步释放

async def use_async_cm():
    print("--- Testing @asynccontextmanager ---")
    async with async_managed_resource("AsyncRes1") as res_name:
        print(f"  Inside async with: Got resource name '{res_name}'", flush=True)
        await asyncio.sleep(0.2)
        print("  Inside async with: Work done.", flush=True)
    print("After async with block.", flush=True)

# 为了在Jupyter中运行asyncio代码
if __name__ == '__main__': # 确保只在直接运行时执行，而不是导入时
    # asyncio.run(use_async_cm()) # 标准运行方式
    # 在Jupyter中，如果已经有事件循环，可能需要特殊处理。
    # 通常，如果IPython版本够新，可以直接await顶层协程。
    # 为了简单演示，这里不直接运行，或者需要nest_asyncio。
    print("Async context manager example defined. Run in an async context.")
    # 如果想在notebook cell中直接运行, 你可以这样做:
    # import nest_asyncio
    # nest_asyncio.apply()
    # asyncio.run(use_async_cm())

## 6. 实际应用场景

上下文管理器在多种场景下都非常有用：

*   **文件操作**：`open()` (最常见的例子)。
*   **锁和同步原语**：`threading.Lock`, `multiprocessing.Lock` 等都支持 `with` 语句，确保锁被正确释放。
    ```python
    # import threading
    # my_lock = threading.Lock()
    # with my_lock:
    #     # 访问共享资源
    #     pass
    ```
*   **数据库连接和事务**：确保连接关闭和事务提交/回滚。
    ```python
    # import sqlite3
    # with sqlite3.connect("mydb.db") as conn: # 连接对象本身就是上下文管理器
    #     cursor = conn.cursor()
    #     # ... 执行SQL ...
    #     conn.commit() # 事务在正常退出时提交，异常时回滚
    ```
*   **网络连接**：确保套接字关闭。
*   **临时改变状态**：例如，临时改变 `decimal` 模块的精度，或临时改变当前工作目录。
    ```python
    # import decimal
    # @contextlib.contextmanager
    # def local_decimal_precision(prec=28):
    #     ctx = decimal.getcontext()
    #     original_prec = ctx.prec
    #     ctx.prec = prec
    #     try:
    #         yield
    #     finally:
    #         ctx.prec = original_prec
    # with local_decimal_precision(50):
    #     # 这里的 decimal 运算使用 50 位精度
    #     pass
    ```
*   **测试装置 (Fixtures)**：在测试中设置和拆卸测试环境。
*   **性能计时和剖析**：如本教程中的 `Timer` 示例。

## 总结

上下文管理器和 `with` 语句是 Python 中进行健壮资源管理和封装设置/拆卸逻辑的关键工具。通过实现上下文管理协议或使用 `contextlib` 模块，你可以编写出更清晰、更可靠的代码。

`contextlib` 模块，特别是 `@contextmanager` 装饰器，极大地简化了自定义上下文管理器的创建过程，使其更加易于使用和理解。